In [18]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/balanced-df/balanced_df.csv


In [19]:
import numpy as np
import pandas as pd
import tensorflow as tf
from transformers import BertTokenizer, TFBertForSequenceClassification
from transformers import AdamW
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

In [20]:
df = pd.read_csv('/kaggle/input/balanced-df/balanced_df.csv')
df.head(-10)

,reviewer_id,store_location,latitude,longitude,date,month,year,title,review,review-label
0,151782.0,US,37.090240,-95.712891,2021,6,2008 00:00:00,I like the tee shirt I ordered butï¿½ï¿½ï¿½ï¿½...,like tee shirt order truth qualiti shirt self ...,1
1,27155.0,US,37.090240,-95.712891,2023,1,2001 00:00:00,Poor quality and smelled bad,order two differ item differ vendor shirt chea...,1
2,72707.0,US,37.090240,-95.712891,2022,8,2007 00:00:00,I did not get my complete order,get complet order im miss mask,1
3,2766.0,US,37.090240,-95.712891,2023,6,2010 00:00:00,I never received the shirt I orderedï¿½ï¿½ï¿½,never receiv shirt order get touch anyon,1
4,240808.0,US,37.090240,-95.712891,2019,9,2028 00:00:00,"Too small, poor quality",normal xl order xl could put long sleev shirt ...,1
...,...,...,...,...,...,...,...,...,...,...
53630,95978.0,CA,56.130366,-106.346771,2022,5,2001 00:00:00,Love it,bought friend one age ago lost arriv within we...,5
53631,188654.0,US,37.090240,-95.712891,2020,9,2023 00:00:00,Fridge magnet and lapel pin,item arriv good item undamag exactli pictur pl...,5
53632,278034.0,CA,56.130366,-106.346771,2018,4,2027 00:00:00,Great quality,great qualiti love shirt even splurg premium o...,5
53633,12182.0,US,37.090240,-95.712891,2023,4,2016 00:00:00,Punctuality was phenomenal,punctual phenomen got everyth expect,5


In [21]:
X = list(df['review'])
Y = df['review-label']

In [22]:
label_encoder = LabelEncoder()
Y = label_encoder.fit_transform(Y)

In [23]:
x_train, x_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=42)
x_train, x_val, Y_train, Y_val = train_test_split(x_train, Y_train, test_size=0.25, random_state=42)

In [24]:
x_train = [str(x) for x in x_train if isinstance(x, (str, int, float))]
x_val = [str(x) for x in x_val if isinstance(x, (str, int, float))]
x_test = [str(x) for x in x_test if isinstance(x, (str, int, float))]

In [25]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

In [26]:
def tokenize_data(texts, tokenizer, max_length=64):
    return tokenizer(
        texts,
        padding=True,
        truncation=True,
        max_length=max_length,
        return_tensors="tf"
    )

In [27]:
train_encodings = tokenize_data(x_train, tokenizer)
val_encodings = tokenize_data(x_val, tokenizer)
test_encodings = tokenize_data(x_test, tokenizer)

In [28]:
train_encodings['input_ids'][602].shape

TensorShape([64])

In [29]:
model = TFBertForSequenceClassification.from_pretrained('bert-base-uncased', 
                                                        num_labels=5, 
                                                        hidden_dropout_prob=0.3)

All PyTorch model weights were used when initializing TFBertForSequenceClassification.

Some weights or buffers of the TF 2.0 model TFBertForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [32]:
optimizer = tf.keras.optimizers.AdamW(learning_rate=5e-5, weight_decay=0.01)

In [33]:
model.compile(optimizer=optimizer,
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

In [34]:
early_stopping = tf.keras.callbacks.EarlyStopping(
    monitor='val_loss',  
    patience=3,           
    restore_best_weights=True
)

In [35]:
history = model.fit(
    train_encodings['input_ids'], np.array(Y_train),
    validation_data=(val_encodings['input_ids'], np.array(Y_val)),
    epochs=10,
    batch_size=32,
    callbacks=[early_stopping]
)

Epoch 1/10
1006/1006 [==============================] - 548s 482ms/step - loss: 1.2363 - accuracy: 0.4557 - val_loss: 1.0991 - val_accuracy: 0.5280
Epoch 2/10
1006/1006 [==============================] - 472s 469ms/step - loss: 1.0576 - accuracy: 0.5441 - val_loss: 1.0505 - val_accuracy: 0.5497
Epoch 3/10
1006/1006 [==============================] - 471s 469ms/step - loss: 0.9945 - accuracy: 0.5761 - val_loss: 1.0503 - val_accuracy: 0.5475
Epoch 4/10
1006/1006 [==============================] - 472s 470ms/step - loss: 0.9453 - accuracy: 0.5960 - val_loss: 1.0474 - val_accuracy: 0.5603
Epoch 5/10
1006/1006 [==============================] - 472s 469ms/step - loss: 0.8946 - accuracy: 0.6175 - val_loss: 1.0949 - val_accuracy: 0.5519
Epoch 6/10
1006/1006 [==============================] - 471s 468ms/step - loss: 0.8447 - accuracy: 0.6443 - val_loss: 1.1055 - val_accuracy: 0.5567
Epoch 7/10
1006/1006 [==============================] - 472s 469ms/step - loss: 0.7909 - accuracy: 0.6732 - val_

In [36]:
# Evaluate the model on validation set
results = model.evaluate(val_encodings['input_ids'], np.array(Y_val))
print(f"Validation Accuracy: {results[1]:.4f}")

336/336 [==============================] - 47s 139ms/step - loss: 1.0474 - accuracy: 0.5603
Validation Accuracy: 0.5603


In [37]:
# Evaluate the model on test set
test_results = model.evaluate(test_encodings['input_ids'], np.array(Y_test))
print(f"Test Accuracy: {test_results[1]:.4f}")

336/336 [==============================] - 47s 139ms/step - loss: 1.0457 - accuracy: 0.5567
Test Accuracy: 0.5567
